In [2]:
import os
import numpy as np
import pandas as pd

In [3]:
#Import CTD bottle nitrate data for calibration of SUNA

infile = './nitratewatersamples.txt'
NO3_bottle = pd.read_csv(infile, sep='\t', names = ["StationNr", "Depth", "Nitrate"])

In [15]:
#Loop through SUNA files
for f_name in os.listdir('./SUNA_profiles'):
    if f_name.endswith('.txt'):
        f_name_elements = f_name.split('sta')
        f_name_getnr = f_name_elements[1].split('.txt')
        infile = './SUNA_profiles/%s' % (f_name)
        SUNA_profile = pd.read_csv(infile, sep=' ')
        #From bin ctd code from Corday
        inc = 1 # define depth increment to bin by (m)
        depthlist = np.arange(0,np.round(np.max(SUNA_profile.Depth), decimals = 0) + inc, inc)
        df = pd.DataFrame(columns=['Station','Depth','NitrateBinned'])
        for j in depthlist:
            #Modify following to read in all profiles
            #loc = df[(df.Cast == i) & (np.round(df.Depth_m, decimals = 0) == j)]
            loc = SUNA_profile[(np.round(SUNA_profile.Depth, decimals = 0) == j)]
            Nitratebinned = np.mean((loc["Nitrate(umol/L)"]))
            #Adjust "5" when looping through files
           # OutputString = "5,%i,%f\n" % (j,Nitratebinned)
            df = df.append({'Station': int(f_name_getnr[0]),'Depth':j,'NitrateBinned':Nitratebinned}, ignore_index=True)

In [16]:
df

,Station,Depth,NitrateBinned
0,5.0,0.0,NaN
1,5.0,1.0,NaN
2,5.0,2.0,NaN
3,5.0,3.0,2.681667
4,5.0,4.0,2.671429
...,...,...,...
363,5.0,363.0,16.520000
364,5.0,364.0,16.598750
365,5.0,365.0,16.431795
366,5.0,366.0,16.392500


In [17]:
#Find corresponding depths between files
for index, row in NO3_bottle.iterrows():
    for indexSUNA, rowSUNA in df.iterrows():
        if int(row['StationNr']) == int(rowSUNA['Station']) and int(row['Depth']) == int(rowSUNA['Depth']):
            print(row['Depth'],' ', row['Nitrate'],' ', rowSUNA['NitrateBinned'])

1.0   0.2471   nan
30.0   0.5347   2.78
150.0   14.1829   nan
200.0   13.1632   15.48


In [ ]:
#Above depth of 150 is returning a nan for the binned nitrate

#Raw unbinned SUNA nitrate data around depth of 150:

#147.515478 15.020000 
#149.549457 NaN 
#151.236311 15.880000 